In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

In [32]:
def generate_str_config(config):
    config_list = []
    
    lr = config['lr_base']
    epochs = config['epochs']
    output_dim = config['output_dim']
    dropout_rate = config['dropout_rate']
    lambdas = config['lambdas']
    top_anomaly_frames = config['top_anomaly_frames']
    top_normal_frames = config['top_normal_frames']
    margin = config['margin']
    
    config_list.append(f'--lr_base {lr}')
    config_list.append(f'--epochs {epochs}')
    config_list.append(f'--output_dim {output_dim}')
    config_list.append(f'--dropout_rate {dropout_rate}')
    config_list.append(f'--lambdas {lambdas}')
    config_list.append(f'--top_anomaly_frames {top_anomaly_frames}')
    config_list.append(f'--top_normal_frames {top_normal_frames}')
    config_list.append(f'--margin {margin}')
    
    config_list.append(f"--exps_dir 'exps/grid_search/triplet_{lr}_{epochs}_{output_dim}_{dropout_rate}_{lambdas}_{top_anomaly_frames}_{top_normal_frames}_{margin}'")
    
    return config_list

def generate_bash_script(config, cuda_devices, generate_func, filename):
    n_gpus = len(cuda_devices)
    full_config = 'cd ..'
    for grid_iter, conf in enumerate(expand_config(config)):
        base_arg = f"CUDA_VISIBLE_DEVICES={cuda_devices[grid_iter%n_gpus]} python TrainingAnomalyDetector_public.py --features_path 'data/anomaly_features' --annotation_path 'Train_Annotation.txt' --network_name 'TripletAnomalyDetector' --objective_name 'triplet_objective_sampling'"

        new_task = copy.deepcopy(base_arg)
        
        res_config = generate_func(conf)
        args = ' '.join(res_config)
        new_task += ' '+args
        
        if (grid_iter+1)%n_gpus!=0: 
            new_task += ' &'
        else:
            new_task += '\nwait'
        full_config += '\n' + new_task if len(full_config) else new_task

    with open (f'./scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [33]:
configs = {
    'lr_base': [0.01, 0.001],
    'epochs': [2, 4, 6],
    'output_dim': [128, 256],
    'dropout_rate': [0.1, 0.5],
    'lambdas': [8e-5, 8e-3],
    'top_anomaly_frames': [3, 5, 10],
    'top_normal_frames': [3, 5, 10],
    'margin': [0.02, 0.2, 2]
}
    
cuda_devices = [1, 3, 7]

generate_bash_script(configs, cuda_devices, generate_str_config, 'grid_search.sh')